In [6]:
from geoband.API import *
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import MarkerCluster, MiniMap
import geojson
import json

## 1. 대전광역시 격자

In [7]:
acci_grid = gpd.read_file('./data/2.대전광역시_교통사고격자(2017~2019).geojson')
grid = acci_grid[['gid', 'geometry']]
# grid.to_file("grid.geojson", driver='GeoJSON') # save

## 2. 연속지적도 상 도로

In [8]:
landmap = gpd.read_file('./data/28.대전광역시_연속지적도.geojson')
landmap.JIBUN = landmap.JIBUN.str[-1:]
landmap = landmap[landmap['JIBUN']=='도']
# landmap.to_file("연속지적도 상 도로.geojson", driver='GeoJSON') # save

## 3. 상세도로망 상 도로 

In [9]:
traffic_link = gpd.read_file('./data/17.대전광역시_교통링크(2018).geojson')
specific_road = gpd.read_file('./data/19.대전광역시_상세도로망(2018).geojson')
traffic_est = pd.read_csv('./data/20.대전광역시_평일_일별_시간대별_추정교통량(2018).csv')

/opt/app-root/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
chaos_frequency = pd.read_csv('./data/21.대전광역시_평일_일별_혼잡빈도강도(2018).csv')
chaos_frequency = chaos_frequency[['상세도로망_LinkID','도로등급','링크길이','혼잡빈도강도']]

chaos_time = pd.read_csv('./data/22.대전광역시_평일_일별_혼잡시간강도(2018).csv')
chaos_time = chaos_time[['상세도로망_LinkID','혼잡시간강도']]

In [11]:
# 상세도로망 데이터와 merge하기 위해 세팅
CI = pd.merge(chaos_time, chaos_frequency, on = '상세도로망_LinkID')
CI = CI[['상세도로망_LinkID','혼잡시간강도','혼잡빈도강도']]
CI['상세도로망_LinkID'] = CI['상세도로망_LinkID'].astype(str)
CI['LinkID'] = CI.상세도로망_LinkID.str[:-2]
CI.drop(['상세도로망_LinkID'],axis=1,inplace = True)

traffic_est = traffic_est[['상세도로망_LinkID','전체_추정교통량']]
traffic_est['상세도로망_LinkID'] = traffic_est['상세도로망_LinkID'].astype(str)
traffic_est['LinkID'] = traffic_est.상세도로망_LinkID.str[:-2]
traffic_est.drop(['상세도로망_LinkID'],axis=1,inplace = True)

In [12]:
result1 = CI.groupby(['LinkID'], as_index = False, sort=False).mean() # 혼잡도
result2 = traffic_est.groupby(['LinkID'],as_index=False, sort=False).mean() # 교통량

In [13]:
# 상세도로망 데이터 셋팅
specific_road = specific_road[['link_id','max_speed','oneway','width','barrier','geometry']]
specific_road.loc[specific_road["barrier"] != '0', 'barrier'] = 1
specific_road.rename({'link_id':'LinkID'}, axis=1, inplace=True)
specific_road['LinkID'] = specific_road['LinkID'].astype(str)

In [14]:
# 상세도로망 데이터와 혼잡도, 교통량 merge
temp = pd.merge(specific_road, result1, on='LinkID',how='outer')
temp = pd.merge(temp, result2,on='LinkID',how='outer')

# 중간값으로 결측 처리
temp['혼잡시간강도'].fillna(74.823,inplace=True)
temp['혼잡빈도강도'].fillna(49.978,inplace=True)
temp['전체_추정교통량'].fillna(3227.5,inplace=True)

In [15]:
result = gpd.sjoin(acci_grid, temp, how='left', op='intersects')
result.dropna(inplace=True)
result.drop('oneway',axis=1,inplace=True)

In [ ]:
# 상세도로망 위 격자
result = gpd.sjoin(acci_grid, temp, how='left', op='intersects')
result.dropna(inplace=True)
result.drop('oneway',axis=1,inplace=True)
result2 = result.groupby(['gid','geometry'],as_index=False,sort=False).mean() # 중복 격자 데이터는 평균으로 처리

result2.to_file("specific_road_to_grid.geojson", driver='GeoJSON') # save

## 파일 로드

In [19]:
grid = gpd.read_file('./grid.geojson') # 대전광역시 격자
landmap = gpd.read_file('./연속지적도 상 도로.geojson') # 연속지적도 상 도로
specific = gpd.read_file('./specific_road_to_grid.geojson') # 상세도로망 상 도로

result = gpd.read_file('./result.geojson') # 전처리 파일

In [21]:
landmap_on_grid = gpd.sjoin(grid, landmap, how='left', op='intersects')
landmap_on_grid.dropna(inplace=True)
print('연속지적도 상 도로 위 격자는 총 {}개입니다.'.format(len(set(landmap_on_grid['gid']))))

연속지적도 상 도로 위 격자는 총 20635개입니다.


In [22]:
grids_on_specific = set(specific['gid'])
print('상세도로망 위 격자는 총 {}개입니다.'.format(len(grids_on_specific)))

상세도로망 위 격자는 총 12411개입니다.


In [24]:
# 연속지적도 위 격자와 상세도로망 위 격자 - 합집합
grids_on_landmap = (set(landmap_on_grid['gid']))
grids_on_road = grids_on_landmap | grids_on_specific
grids_on_road = list(grids_on_road)
print('도로 위 격자는 총 {}개입니다.'.format(len(grids_on_road)))

# 도로 위 격자
result_on_road = result[result['gid'].isin(grids_on_road)]

도로 위 격자는 총 21449개입니다.


In [27]:
# 나머지 데이터에 사고 건수가 있다면 도로 위가 아니더라도 격자에 포함한다
result_noton_road = result[~result['gid'].isin(grids_on_road)]
temp = result_noton_road[result_noton_road['acci_cnt']>0]
grids_acci = list(temp['gid'])

In [28]:
# 최종 전처리 파일
total_grids = grids_acci + grids_on_road
final_result = result[result['gid'].isin(total_grids)]

In [32]:
final_result.to_file("final_result.geojson", driver='GeoJSON') # geojson 파일로 저장
final_result.to_csv("./final_result.csv") # csv 파일로 저장